## ДЗ №3 Двухуровневый пайплайн
#### В этой домашке вам предстоит написать с нуля двустадийную рекомендательную систему. 

#### Дата выдачи: 10.03.25

#### Мягкий дедлайн: 31.03.25 23:59 MSK

#### Жесткий дедлайн: 7.04.25 23:59 MSK

### Описание
Это творческое задание, в котором вам необходимо реализовать полный цикл построения рекомендательной системы: реализовать кандидат генераторов, придумать и собрать признаки, обучить итоговый ранкер и заинференсить модели на всех пользователей.

Вам предоставляется два набора данных: `train.csv` и `test.csv` 

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import implicit
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

In [3]:
# скачиваем данные
# если из этой ячейки не получается, то вот ссылка на папку https://drive.google.com/drive/folders/1HT0Apm8Jft0VPLJtdBBUGu9s1M7vZcoJ?usp=drive_link

!pip3 install gdown


import gdown
# train
url = "https://drive.google.com/file/d/1-CcS22-UpTJeNcFlA0dVLrEQn8jnI0d-/view?usp=drive_link"
output = 'train.csv'
gdown.download(url, output, quiet=False)

# test
url = "https://drive.google.com/file/d/11iz3xDh0IIoEIBY0dyRSvByY3qfiT3BG/view?usp=drive_link"
output = 'test.csv'
gdown.download(url, output, quiet=False)

d:\HSE\Recsis_1(3-4)\DZ3\.venv\lib\site-packages\gdown\parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1-CcS22-UpTJeNcFlA0dVLrEQn8jnI0d-
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1-CcS22-UpTJeNcFlA0dVLrEQn8jnI0d-/view?usp=drive_link
To: d:\HSE\Recsis_1(3-4)\DZ3\train.csv
91.5kB [00:00, 3.23MB/s]
d:\HSE\Recsis_1(3-4)\DZ3\.venv\lib\site-packages\gdown\parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=11iz3xDh0IIoEIBY0dyRSvByY3qfiT3BG
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/11iz3xDh0IIoEIBY0dyRSvByY3qfiT3BG/view?usp=drive_link
To: d:\HSE\Recsis_1(3-4)\DZ3\test.csv
91.6kB [00:00, 3.17MB/s]


'test.csv'



### 1 Этап. Модели первого уровня. (max 3 балла)
В этом этапе вам необходимо разделить `train` датасет на 2 части: для обучения моделей первого уровня и для их валидации. Единственное условие для разбиения – разбивать нужно по времени. Данные для обучение будем называть `train_stage_1`, данные для валидации `valid_stage_1`. Объемы этих датасетов вы определяет самостоятельно. 

Для начала нам нужно отобрать кандидатов при помощи легких моделей. Необходимо реализовать 3 типа моделей:
1. Любая эвристическая(алгоритмичная) модель на ваш выбор **(0.5 балл)**
2. Любая матричная факторизация на ваш выбор **(1 балл)**
3. Любая нейросетевая модель на ваш выбор **(1 балла)**

Не забудьте использовать скор каждой модели, как признак!



In [5]:
# Загрузка данных
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.last_watch_dt.max()

'2021-08-12'

In [16]:
# Преобразование временных меток
train_df['weekday'] = pd.to_datetime(train_df.last_watch_dt).dt.weekday
test_df['weekday'] = pd.to_datetime(test_df.last_watch_dt).dt.weekday
train_df = train_df.sort_values('last_watch_dt')

train_stage_1 = train_df.loc[(train_df.last_watch_dt < '2021-08-06')].copy()
valid_stage_1 = train_df.loc[train_df.last_watch_dt >= '2021-08-06'].copy()

user_ids = train_df['user_id'].unique()
item_ids = train_df['item_id'].unique()
user_to_idx = {user: idx for idx, user in enumerate(user_ids)}
idx_to_user = {idx: user for user, idx in user_to_idx.items()}
item_to_idx = {item: idx for idx, item in enumerate(item_ids)}
idx_to_item = {idx: item for item, idx in item_to_idx.items()}

In [17]:
test_df.shape, train_stage_1.shape, valid_stage_1.shape

((608467, 7), (4473822, 7), (393134, 7))

In [18]:
# 1. Эвристическая модель (популярность по дням недели)
class HeuristicModel:
    def __init__(self):
        self.popularity = defaultdict(dict)
        self.default_score = 0.5
        
    def fit(self, df):
        for weekday in range(7):
            weekday_data = df[df['weekday'] == weekday]
            item_counts = weekday_data['item_id'].value_counts()
            total = item_counts.sum()
            for item, count in item_counts.items():
                self.popularity[weekday][item] = count / total
    
    def predict(self, user_id, item_id, weekday=None, default=None):
        if weekday is None:
            weekday = np.random.randint(0, 7)
            
        if item_id in self.popularity[weekday]:
            return self.popularity[weekday][item_id]
        return default if default is not None else self.default_score
    
    def recommend(self, user_id, k=10, weekday=None):
        if weekday is None:
            weekday = np.random.randint(0, 7)
            
        popular_items = sorted(self.popularity[weekday].items(), 
                             key=lambda x: -x[1])
        return [item for item, score in popular_items[:k]]

In [19]:
# 2. Матричная факторизация (implicit ALS)
class ImplicitFactorizationModel:
    def __init__(self, factors=50, iterations=15, regularization=0.01):
        self.model = implicit.als.AlternatingLeastSquares(
            factors=factors,
            iterations=iterations,
            regularization=regularization,
            random_state=42
        )
        self.default_score = 0.5
        self.user_factors = None
        self.item_factors = None
        
    def fit(self, df):
        # Создаем разреженную матрицу взаимодействий
        user_indices = df['user_id'].map(user_to_idx).values
        item_indices = df['item_id'].map(item_to_idx).values
        durations = df['total_dur'].values
        
        # Нормализуем продолжительности
        durations = durations / durations.max()
        
        # Создаем матрицу в формате COO
        from scipy.sparse import coo_matrix
        interactions = coo_matrix(
            (durations, (user_indices, item_indices)),
            shape=(len(user_to_idx), len(item_to_idx))
        )
        
        # Обучаем модель
        self.model.fit(interactions)
        self.user_factors = self.model.user_factors
        self.item_factors = self.model.item_factors
        
    def predict(self, user_id, item_id, default=None):
        if user_id not in user_to_idx or item_id not in item_to_idx:
            return default if default is not None else self.default_score
            
        user_idx = user_to_idx[user_id]
        item_idx = item_to_idx[item_id]
        return self.user_factors[user_idx] @ self.item_factors[item_idx].T
    
    def recommend(self, user_id, k=10):
        if user_id not in user_to_idx:
            return []
            
        user_idx = user_to_idx[user_id]
        items, scores = self.model.recommend(
            user_idx, 
            user_items=None, 
            N=k, 
            filter_already_liked_items=False
        )
        return [idx_to_item[item] for item in items]

In [20]:
# 3. Нейросетевая модель (PyTorch)
class InteractionDataset(Dataset):
    def __init__(self, df, user_to_idx, item_to_idx):
        self.df = df.copy()
        self.df['user_idx'] = self.df['user_id'].map(user_to_idx)
        self.df['item_idx'] = self.df['item_id'].map(item_to_idx)
        self.df = self.df.dropna(subset=['user_idx', 'item_idx'])
        
        # Нормализуем target
        self.scaler = MinMaxScaler()
        self.df['target'] = self.scaler.fit_transform(
            self.df[['total_dur']]
        )
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        return (
            torch.LongTensor([row['user_idx']]),
            torch.LongTensor([row['item_idx']]),
            torch.FloatTensor([row['target']])
        )

class NeuralCF(nn.Module):
    def __init__(self, n_users, n_items, emb_dim=64):
        super().__init__()
        self.user_embedding = nn.Embedding(n_users, emb_dim)
        self.item_embedding = nn.Embedding(n_items, emb_dim)
        self.fc = nn.Sequential(
            nn.Linear(emb_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        
    def forward(self, user, item):
        user_emb = self.user_embedding(user).squeeze(1)
        item_emb = self.item_embedding(item).squeeze(1)
        x = torch.cat([user_emb, item_emb], dim=1)
        return self.fc(x)

class TorchNeuralModel:
    def __init__(self, emb_dim=64, lr=0.001, batch_size=256, n_epochs=5):
        self.model = None
        self.emb_dim = emb_dim
        self.lr = lr
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.default_score = 0.5
        self.scaler = MinMaxScaler()
        
    def fit(self, df):
        # Подготовка данных
        dataset = InteractionDataset(df, user_to_idx, item_to_idx)
        self.scaler = dataset.scaler
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        
        # Инициализация модели
        n_users = len(user_to_idx)
        n_items = len(item_to_idx)
        self.model = NeuralCF(n_users, n_items, self.emb_dim)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        
        # Обучение
        for epoch in range(self.n_epochs):
            for user, item, target in dataloader:
                optimizer.zero_grad()
                output = self.model(user, item)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
            print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')
    
    def predict(self, user_id, item_id, default=None):
        if user_id not in user_to_idx or item_id not in item_to_idx:
            return default if default is not None else self.default_score
            
        user_idx = torch.LongTensor([user_to_idx[user_id]])
        item_idx = torch.LongTensor([item_to_idx[item_id]])
        with torch.no_grad():
            score = self.model(user_idx, item_idx).item()
        return score
    
    def recommend(self, user_id, k=10):
        if user_id not in user_to_idx:
            return []
            
        user_idx = user_to_idx[user_id]
        user_tensor = torch.LongTensor([user_idx] * len(item_to_idx))
        item_tensor = torch.LongTensor(list(item_to_idx.values()))
        
        with torch.no_grad():
            scores = self.model(user_tensor, item_tensor).flatten().numpy()
        
        top_items = np.argsort(-scores)[:k]
        return [idx_to_item[item] for item in top_items]

In [ ]:
# Инициализация и обучение моделей
print("Training Heuristic Model...")
heuristic_model = HeuristicModel()
heuristic_model.fit(train_stage_1)
print("Training Implicit ALS Model...")
implicit_model = ImplicitFactorizationModel(factors=50, iterations=15)
implicit_model.fit(train_stage_1)
print("Training PyTorch Neural Model...")
torch_model = TorchNeuralModel(emb_dim=64, n_epochs=5)
torch_model.fit(train_stage_1)

Training Heuristic Model...


In [23]:
print("Training Implicit ALS Model...")
implicit_model = ImplicitFactorizationModel(factors=50, iterations=15)
implicit_model.fit(train_stage_1)

Training Implicit ALS Model...


d:\HSE\Recsis_1(3-4)\DZ3\.venv\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3573026657104492 seconds
  warnings.warn(
100%|██████████| 15/15 [00:57<00:00,  3.81s/it]


In [25]:
print("Training PyTorch Neural Model...")
torch_model = TorchNeuralModel(emb_dim=64, n_epochs=5)
torch_model.fit(train_stage_1)

Training PyTorch Neural Model...


KeyboardInterrupt: 

In [ ]:
# Функция для оценки разнообразия рекомендаций
def evaluate_diversity(models, users, k=10):
    overlaps = np.zeros((len(models), len(models)))
    diversities = []
    
    for user in users[:1000]:  # Ограничим выборку для скорости
        recommendations = []
        for model in models:
            try:
                recs = set(model.recommend(user, k=k))
                recommendations.append(recs)
            except:
                recommendations.append(set())
        
        # Рассчитываем попарные пересечения
        for i in range(len(models)):
            for j in range(len(models)):
                overlaps[i,j] += len(recommendations[i] & recommendations[j]) / k
                
        # Рассчитываем diversity для одного пользователя
        all_recs = set()
        for rec in recommendations:
            all_recs.update(rec)
        diversity = len(all_recs) / (len(models) * k)
        diversities.append(diversity)
    
    overlaps /= min(1000, len(users))
    mean_diversity = np.mean(diversities)
    
    print("Pairwise Overlap Matrix:")
    print(overlaps)
    print(f"\nMean Diversity: {mean_diversity:.3f}")
    
    return overlaps, mean_diversity

# Оценка разнообразия рекомендаций
print("\nEvaluating Recommendation Diversity...")
models = [heuristic_model, implicit_model, torch_model]
sample_users = valid_stage_1['user_id'].unique()[:1000]
overlaps, diversity = evaluate_diversity(models, sample_users)

Каждая модель должна уметь:
1) для пары user_item предсказывать скор релевантности (масштаб скора не важен), важно обработать случаи, когда модель не можеn проскорить пользователя или айтем, вместо этого вернуть какое-то дефолтное значение
2) для всех пользователей вернуть top-k самых релевантных айтемов (тут вам скоры не нужны)


Дополнительно можно провести анализ кандидат генератов, измерить насколько различные айтемы они рекомендуют, например с помощью таких метрик как: [Ranked based overlap](https://github.com/changyaochen/rbo) или различные вариации [Diversity](https://github.com/MaurizioFD/RecSys2019_DeepLearning_Evaluation/blob/master/Base/Evaluation/metrics.py#L289). **(1 балл)**


### 2 Этап. Генерация и сборка признаков. (max 2 балла)
Необходимо собрать минимум 10 осмысленных (`np.radndom.rand()` не подойдет) признаков, при этом:
1. 2 должны относиться только к сущности "пользователь" (например средний % просмотра фильмов у этой возрастной категории)
2. 2 должны относиться только к сущности "айтем" (например средний средний % просмотра данного фильма)
3. 6 признаков, которые показывают связь пользователя и айтема (например средний % просмотра фильмов с данным актером (айтем) у пользователей с таким же полом (пользователь)). 

### ВАЖНО!  

1. **В датасете есть колонка `watched_prct`. Ее можно использовать для генерации признаков (например сколько пользователь в среднем смотрит фильмы), но нельзя подавать в модель, как отдельную фичу, потому что она напрямую связана с target.**
2. **Все признаки должны быть собраны без дата лика, то есть если пользователь посмотрел фильм 10 августа, то признаки мы можем считать только на данных до 9 августа включительно.**


### Разбалловка
Обучение ранкера будет проходить на `valid_stage_1`, как  раз на которой мы валидировали модели, а тестировать на `test`. Поэтому есть 2 варианта сборки признаков, **реализовать нужно только 1 из них:**
1. Для обучения собираем признаки на первый день `valid_stage_1`, а для теста на первый день `test`. Например, если `valid_stage_1` начинается 5 сентября, то все признаки мы можем собирать только по 4 сентября включительно. **(1 балл)**
2. Признаки будем собирать честно на каждый день, то есть на 5 сентября собираем с начала до 4, на 6 сентября с начала до 5 и т.д. **(2 балла)**

In [ ]:
train_df_with_features = # YOUR CODE IS HERE
test_df_with_features = # YOUR CODE IS HERE


### 3 Этап. Обучение финального ранкера (max 2 балла)
Собрав все признаки из этапа 2, добавив скоры моделей из этапа 1 для каждой пары пользователь-айтем (где это возможно), пришло время обучать ранкер. В качестве ранкера можно использовать либо [xgboost](https://xgboost.readthedocs.io/en/stable/) или [catboost](https://catboost.ai/). Обучать можно как `Classfier`, так и `Ranker`, выбираем то, что лучше сработает. Обучение ранкера будет проходить на `valid_stage_1`, как  раз на которой мы валидировали модели, а тестировать на `test`, которую мы до сих пор не трогали.  Заметьте, что у нас в тесте есть холодные пользователи – те, кого не было в train и активные – те, кто был в train. Возможно их стоит обработать по отдельности (а может и нет).  
(1 балл)

После получения лучшей модели надо посмотреть на важность признаков и [shap values](https://shap.readthedocs.io/en/latest/index.html), чтобы:
1. Интерпритировать признаки, которые вы собрали, насколько они полезные
2. Проверить наличие ликов – если важность фичи в 100 раз больше, чем у всех остальных, то явно что-то не то  

(1 балл)






In [ ]:
# YOUR FIT PREDICT CODE HERE
model.fit()
model.predict()


### 4 Этап. Инференс лучшего ранкера (max 3 балла)

Теперь мы хотим построить рекомендации "на завтра", для этого нам нужно:

1. Обучить модели первого уровня на всех (train+test) данных (0.5 балла)
2. Для каждой модели первого уровня для каждого пользователя сгененировать N кандидатов (0.5 балла)
3. "Склеить" всех кандидатов для каждого пользователя (дубли выкинуть), посчитать скоры от всех моделей (0.5 балла)
4. Собрать фичи для ваших кандидатов (теперь можем считать признаки на всех данных) (0.5 балла)
5. Проскорить всех кандидатов бустингом и оставить k лучших (0.5 балла)
6. Посчитать разнообразие(Diversity) и построить график от Diversity(k) (0.5 балла)


Все гиперпараметры (N, k) определяете только Вы!

In [3]:
# YOUR CODE HERE